### Python & Banco de Dados

O passo a passo é para Windows. Caso tenha outro sistema operacional, o objetivo é ter o python e banco de dados na sua máquina <br><br>

1 - Instalar o Anaconda para ter uma plataforma Python Local: https://www.anaconda.com/<br>
2 - Não é necessário marcar nenhuma opção durante a instalação<br>
3 - Ao terminar a instalação, abrir o Jupyter Notebook<br>
4 - Executar o comando pip install psycopg2. Mais orientações em https://pypi.org/project/psycopg2/


In [4]:
pip install psycopg2


5 - Considerando o banco de dados utilizado desde a primeira aula (dvdrental) faça a conexão abaixo

In [81]:
import psycopg2
#conn = psycopg2.connect("dbname=dvdrental user=postgres password=admin")

# ou pode ser feito como abaixo:
conn = psycopg2.connect(
    host="localhost", 
    database="kaggle",
    user="postgres",
    password="admin")

In [74]:
import pandas as pd
import numpy as np

df = pd.read_csv("ds_salaries.csv", index_col=False )

df.head()

,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


In [75]:
df.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  607 non-null    int64 
 1   work_year           607 non-null    int64 
 2   experience_level    607 non-null    object
 3   employment_type     607 non-null    object
 4   job_title           607 non-null    object
 5   salary              607 non-null    int64 
 6   salary_currency     607 non-null    object
 7   salary_in_usd       607 non-null    int64 
 8   employee_residence  607 non-null    object
 9   remote_ratio        607 non-null    int64 
 10  company_location    607 non-null    object
 11  company_size        607 non-null    object
dtypes: int64(5), object(7)
memory usage: 57.0+ KB


In [99]:
cols = ",".join([str(i) + " varchar(100)" for i in df.columns.tolist()])
cols

'id varchar(100),work_year varchar(100),experience_level varchar(100),employment_type varchar(100),job_title varchar(100),salary varchar(100),salary_currency varchar(100),salary_in_usd varchar(100),employee_residence varchar(100),remote_ratio varchar(100),company_location varchar(100),company_size varchar(100)'

In [103]:
conn = psycopg2.connect(
    host="localhost", 
    database="kaggle",
    user="postgres",
    password="admin")
cur = conn.cursor()

print("Criando um banco")

try:
    cols = ",".join([str(i) + " varchar(100)" for i in df.columns.tolist()])
    cur.execute(f"CREATE TEMPORARY TABLE salary({cols})")   
    
    
    
    cols = ",".join([str(i) for i in df.columns.tolist()])
    for i,row in df.iterrows():
    #cur = conn.cursor()
        sql = "INSERT INTO salary (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        #print(sql)
        cur.execute(sql, tuple(row))
        #print(tuple(row))

    sql = f"""CREATE TABLE IF NOT EXISTS SALARY_FINAL
            AS select 
                    CAST(id AS INT)  as ID
                    ,COALESCE(cast(work_year as smallint), 2020) AS WORK_YEAR
                    ,CAST(company_location as varchar(2))  AS company_location
            from salary;"""

    cur.execute(sql)        
 
        
    conn.commit() #alteracoes no banco precisam deste comando para formalizar a execucao
    # bom evitar a conexão aberta.
    cur.close()
    
    print("Banco criado com sucesso")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
        print('Conexão encerrada.')

Criando um banco
Banco criado com sucesso
Conexão encerrada.


In [96]:
conn.close()
cur.close()